# Validierung

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support


# Einstellungen

In [2]:
# --- Einstellungen ---
MODEL_PATH = r"C:\Users\jonni\OneDrive\Dokumente\GitHub\AI-Training-Jupyter\runs\train\yolo11n_custom\weights\best.pt"
TEST_IMAGES = r"C:\Users\jonni\OneDrive\Dokumente\GitHub\Chairlift_Gefahrenerkennung\01_Data_Jon\Augmented\Images_split\Images\train\images"    # Test Verzeichnis auswählen
HYPERPARAMETERS = r"C:\Users\jonni\OneDrive\Dokumente\GitHub\AI-Training-Jupyter\conf_iou_tuning_per_class.csv"

# Anzahl der Bilder printen
print(f"Anzahl der Bilder zur Validierung: {len(os.listdir(TEST_IMAGES))}")

Anzahl der Bilder zur Validierung: 315


Das trainierte Modell hat diese Test-Bilder noch nie "gesehen".

In [4]:
# CLASS_NAMES aus dem YOLO Modell extrahieren
MODEL = YOLO(MODEL_PATH)
CLASS_NAMES = MODEL.names
N_CLASSES = len(CLASS_NAMES)

print(f"Geladene Klassen: {CLASS_NAMES}")

Geladene Klassen: {0: 'KLEBER', 1: 'EDDING', 2: 'TACKER'}


In [5]:
# Trennzeichen automatisch erkennen
try:
    tuning = pd.read_csv(HYPERPARAMETERS, sep=";")
except Exception:
    tuning = pd.read_csv(HYPERPARAMETERS, sep=",")
if 'Klasse' not in tuning.columns:
    tuning.rename(columns={tuning.columns[0]: 'Klasse'}, inplace=True)
print(f"Gefundene Klassen in Hyperparameter-CSV: {tuning['Klasse'].tolist()}")

CLASS_NAMES = tuning['Klasse'].tolist()
N_CLASSES = len(CLASS_NAMES)

conf_threshs = dict(zip(tuning['Klasse'], tuning['Best_Conf']))
iou_threshs = dict(zip(tuning['Klasse'], tuning['Best_IoU']))

# --- Hilfsfunktion für Labelpfad (systemunabhängig) ---
def get_label_path(img_path):
    dir_path, file_name = os.path.split(img_path)
    dir_path_labels = dir_path.replace(f"{os.sep}images", f"{os.sep}labels")
    base, _ = os.path.splitext(file_name)
    label_path = os.path.join(dir_path_labels, base + ".txt")
    return label_path

# --- Vorhersagen auf Testdatensatz ---
model = YOLO(MODEL_PATH)
results = model.predict(TEST_IMAGES, save=False, conf=0.01, iou=0.01, stream=True, verbose=False)

y_true, y_pred = [], []

for r in results:
    img_path = r.path
    boxes = r.boxes.xyxy.cpu().numpy() if r.boxes is not None else np.zeros((0,4))
    scores = r.boxes.conf.cpu().numpy() if r.boxes is not None else np.zeros((0,))
    classes = r.boxes.cls.cpu().numpy().astype(int) if r.boxes is not None else np.zeros((0,))
    label_path = get_label_path(img_path)
    gts = []
    if os.path.isfile(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                cls, x, y, w, h = int(float(parts[0])), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                img = plt.imread(img_path)
                h_img, w_img = img.shape[:2]
                cx, cy = x*w_img, y*h_img
                bw, bh = w*w_img, h*h_img
                x1, y1 = max(0, cx-bw/2), max(0, cy-bh/2)
                x2, y2 = min(w_img, cx+bw/2), min(h_img, cy+bh/2)
                gts.append([cls, x1, y1, x2, y2])
    gts = np.array(gts)

    for k, name in enumerate(CLASS_NAMES):
        conf_thr = float(conf_threshs[name])
        iou_thr = float(iou_threshs[name])
        keep = (classes == k) & (scores >= conf_thr)
        pred_boxes = boxes[keep]
        gt_boxes = gts[gts[:,0]==k][:,1:] if gts.size>0 else np.empty((0,4))
        used_gt = set()

        # Für jede Prediction: Ist es ein Match (TP), sonst FP
        for pb in pred_boxes:
            best_iou = 0
            best_j = -1
            for j, gb in enumerate(gt_boxes):
                if j in used_gt:
                    continue
                xA = max(pb[0], gb[0])
                yA = max(pb[1], gb[1])
                xB = min(pb[2], gb[2])
                yB = min(pb[3], gb[3])
                interArea = max(0, xB - xA) * max(0, yB - yA)
                pb_area = max(1e-8, (pb[2]-pb[0])*(pb[3]-pb[1]))
                gb_area = max(1e-8, (gb[2]-gb[0])*(gb[3]-gb[1]))
                iou = interArea / float(pb_area + gb_area - interArea + 1e-8)
                if iou > best_iou:
                    best_iou = iou
                    best_j = j
            if best_iou >= iou_thr and best_j >= 0:
                y_true.append(k)
                y_pred.append(k)
                used_gt.add(best_j)
            else:
                y_true.append(N_CLASSES)     # Dummyklasse für "kein Objekt"
                y_pred.append(k)             # Falsch erkannt als k

        # Für jede nicht gematchte GT-Box: FN
        for j in range(len(gt_boxes)):
            if j not in used_gt:
                y_true.append(k)
                y_pred.append(N_CLASSES)     # Nicht erkannt

# Dummyklasse "Hintergrund/Nicht erkannt"
if any([x==N_CLASSES for x in y_true+y_pred]):
    full_names = CLASS_NAMES + ["Nicht erkannt"]
else:
    full_names = CLASS_NAMES

# Jetzt ist y_true/y_pred exakt gleich lang!
report = classification_report(y_true, y_pred, target_names=full_names, digits=3, output_dict=True, zero_division=0)
df_report = pd.DataFrame(report).transpose()

cm = confusion_matrix(y_true, y_pred, labels=list(range(len(full_names))))

# --- Visualisierung: Confusion Matrix ---
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(np.arange(len(CLASS_NAMES)))
ax.set_yticks(np.arange(len(CLASS_NAMES)))
ax.set_xticklabels(CLASS_NAMES, rotation=45, ha="right")
ax.set_yticklabels(CLASS_NAMES)
plt.xlabel("Vorhergesagte Klasse")
plt.ylabel("Tatsächliche Klasse")
plt.title("Confusion Matrix – Test-Datensatz")
plt.colorbar(im)
for i in range(len(CLASS_NAMES)):
    for j in range(len(CLASS_NAMES)):
        ax.text(j, i, cm[i, j], ha="center", va="center", color="black")
plt.tight_layout()
plt.savefig("PLOT_validation_confusion_matrix.png")
plt.close()

print("Validierungsbericht und Visualisierung erstellt (Markdown + PNG).")


Gefundene Klassen in Hyperparameter-CSV: ['KLEBER', 'EDDING', 'TACKER']
Validierungsbericht und Visualisierung erstellt (Markdown + PNG).
